In [1]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

import tqdm 
import tqdm.notebook
import scipy 
import pandas as pd
# temp: functions and more...
from optimization import *
import optimization_clean as opt_clean

%load_ext autoreload
%autoreload 2

### How do the values $\mathbb{E}_{\mathcal{G}} [|\mathcal{S}^{\ast}(\lambda)|]$ and $\mathbb{E}_{\mathcal{G}} [f(\lambda, \mathcal{S}^{\ast}(\lambda))]$ evolve with respect to $\lambda$?

For now we fix $\beta = 10$, to run this experiment.

In [5]:
n_iter = 10000
def beta(i, n_iter):
    if i < n_iter * (1/5) :
        return 1
    elif i < n_iter * (2/5):
        return 5
    elif i < n_iter * (3/5):
        return 10
    elif i < n_iter * (4/5):
        return 20
    else: 
        return 50

In [6]:
def effect_of_lambda(n_iter, beta, generator, N, lambda_range, num_runs, mutation_strategy, init_prob):
    # dictionary to save the results
    results_f = {l: [] for l in lambda_range}
    results_num_cities = {l: [] for l in lambda_range}
    for i in range(num_runs):
        cities = generator(N)
        for l in lambda_range:
            if mutation_strategy == 5:
                sel_c_conv, loss_convex = opt_clean.optimize(
                        cities, l, beta=beta, n_iter=n_iter, verbose=False, use_kd_tree=True)
            else:
                sel_c, sel_c_conv, loss_values, loss_convex = optimize(cities, l, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=mutation_strategy,
                                          initial_selection_probability=init_prob,
                                          precompute_pairwise_dist=False, verbose=False)
            # save the result
            results_f[l].append(loss_convex)
            if mutation_strategy == 5:
                results_num_cities[l].append(sum(sel_c_conv[-1]))
            else:
                results_num_cities[l].append(sum(sel_c_conv))
    return results_num_cities, results_f

In [7]:
N = 1000
num_iter = 1000
num_runs = 1
lambda_range = [0, 0.2, 0.4, 0.6, 0.8, 1]
# mutation_strategy=3
init_prob=0.0

r_G1_num_cities={0: [], 1: [], 3: [], 5: []}
r_G1_f={0: [], 1: [], 3: [], 5: []}

r_G2_num_cities={0: [], 1: [], 3: [], 5: []}
r_G2_f={0: [], 1: [], 3: [], 5: []}
for i in [0, 1, 3, 5]:
    r_G1_num_cities[i], r_G1_f[i] =  effect_of_lambda(num_iter, beta, G1, N, lambda_range, num_runs, mutation_strategy=i, init_prob=init_prob)

    r_G2_num_cities[i], r_G2_f[i] =  effect_of_lambda(num_iter, beta, G2, N, lambda_range, num_runs, mutation_strategy=i, init_prob=init_prob)
    print("strategy {} finished".format(i))

strategy 0 finished
strategy 1 finished
strategy 3 finished
Convex  -4.445303220413687  vs  -4.445303220413688
Convex  -119.96767111098177  vs  -119.9676711109818
Convex  -48.08889756137471  vs  -48.088897561374736
strategy 5 finished


In [8]:
r_G1_num_cities

{0: {0: [988], 0.2: [18], 0.4: [5], 0.6: [3], 0.8: [3], 1: [1]},
 1: {0: [981], 0.2: [16], 0.4: [7], 0.6: [1], 0.8: [2], 1: [1]},
 2: [],
 3: {0: [874], 0.2: [885], 0.4: [10], 0.6: [4], 0.8: [2], 1: [1]},
 5: {0: [1000], 0.2: [899], 0.4: [834], 0.6: [40], 0.8: [8], 1: [7]}}

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))

fig.text(0.3, 1, r"Effect of $\lambda$. Results are for {} runs".format(num_runs), fontsize=12)
for i in [0, 1, 3, 5]:
    ax[0][0].plot(lambda_range, [np.mean(i) for i in r_G1_num_cities[i].values()], label="strategy {}".format(i))
    ax[0][0].set_title(r"$\mathbb{E}_{\mathcal{G}} [|\mathcal{S}^{\ast}(\lambda)|]$ for uniform generator $\mathcal{G}_1$")
    ax[0][0].legend()

for i in [0, 1, 3, 5]:
    ax[0][1].plot(lambda_range, [np.mean(i) for i in r_G2_num_cities[i].values()], label="strategy {}".format(i))
    ax[0][1].set_title(r"$\mathbb{E}_{\mathcal{G}} [|\mathcal{S}^{\ast}(\lambda)|]$ for log_normal generator $\mathcal{G}_2$")
    ax[0][1].legend()

for i in [0, 1, 3, 5]:
    ax[1][0].plot(lambda_range, [np.mean(i) for i in r_G1_f[i].values()], label="strategy {}".format(i))
    ax[1][0].set_title(r"$\mathbb{E}_{\mathcal{G}} [f(\lambda, \mathcal{S}^{\ast}(\lambda))]$ for uniform generator $\mathcal{G}_1$")
    ax[1][0].legend()
    
for i in [0, 1, 3, 5]:
    ax[1][1].plot(lambda_range, [np.mean(i) for i in r_G2_f[i].values()], label="strategy {}".format(i))
    ax[1][1].set_title(r"$\mathbb{E}_{\mathcal{G}} [f(\lambda, \mathcal{S}^{\ast}(\lambda))]$ for log-normal generator $\mathcal{G}_2$")
    ax[1][1].legend()
    
ax[1][0].set_xlabel(r"$\lambda$")
ax[1][1].set_xlabel(r"$\lambda$")
ax[0][0].set_ylabel(r"$\mathbb{E}_{\mathcal{G}} [|\mathcal{S}^{\ast}(\lambda)|]$", fontsize=12)
ax[1][0].set_ylabel(r"$\mathbb{E}_{\mathcal{G}} [f(\lambda, \mathcal{S}^{\ast}(\lambda))]$", fontsize=12)
fig.savefig("Expectation_f_num_cities_{}.pdf".format(N), dpi=1200);

### Effect of $\beta$ (deprecated for now)

For a fixed $\lambda$ and different initializations of the selected cities, how does the value of value of $\beta$ affects the minimum value acheived for the objective function?

In [ ]:
def effect_of_beta(n_iter, l, cities, beta_range, num_runs, mutation_strategy, 
                   init_prob):
    # dictionary to save the results
    results = {beta: [] for beta in beta_range}
    l = 0.8
    for beta in beta_range:
        np.random.seed(2)
        for i in range(num_runs):
            sel_c, sel_conv, loss, loss_convex = optimize(cities, l, beta=beta, 
                                          n_iter=n_iter, mutation_strategy=mutation_strategy,
                                          initial_selection_probability=init_prob,
                                          precompute_pairwise_dist=False, verbose=False)
            # save the result
            results[beta].append(loss_convex)
    return results

In [ ]:
N = 100
num_iter = 5000
num_runs = 10
l = 0.4
beta_range = [5, 10, 25, 50, 100, 1000]
g = G1(N)
mutation_strategy=3
init_prob=1/N
r_G1 = effect_of_beta(num_iter, l, g, beta_range, num_runs, mutation_strategy, init_prob)

g = G2(N)
r_G2 = effect_of_beta(num_iter, l, g, beta_range, num_runs, mutation_strategy, init_prob)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
df = pd.DataFrame(r_G1)
fig.text(0.3, 1, r"Effect of $\beta$. Results are for {} runs when $\lambda$ is fixed to {}".format(num_runs, l), fontsize=12)
df.plot(ax = ax[0], kind="box", title= r"uniform generator $\mathcal{G}_1$")
df = pd.DataFrame(r_G2)
df.plot(ax = ax[1], kind="box", title= r"log-normal generator $\mathcal{G}_2$")
ax[1].set_xlabel(r"$\beta$")
ax[0].set_xlabel(r"$\beta$")
ax[0].set_ylabel("min(f)");